In [338]:
#1.模块导入
import dolphindb as ddb
import numpy as np
import pandas as pd
import os
from multiprocessing import Queue, Process
import time
import random

In [339]:
#2.参数配置
# parallel 是指定并发数（并发查询进程数）
parallels = [100]
# 单个进程执行多少次循环查询
sqlNum = 100
# DolphinDB 测试节点信息
ddbIP = "ip"
ddbPorts = [8073, 8074]
ddbUserId = "admin"
ddbUserPassword = "123456"
dbName = "dfs://l2TLDB"
tbName = "trade"

In [348]:
#3.定义查询函数
# 单个进程的查询函数
countQue = Queue()
costQue = Queue()
def testSQL(processName, sql):
    s = ddb.session(compress=True)
    ddbPort = ddbPorts[random.sample(range(0,2), 1)[0]]
    s.connect(host=ddbIP, port=ddbPort, userid=ddbUserId, password=ddbUserPassword)
    #print(f"The query of process {os.getpid()} begins, the start time is：{pd.Timestamp.now()},"+"IP:'{ip}'".format(ip=ddbIP) , flush=True)
    count = 0
    cost = 0
    for i in range(sqlNum):
        startTime = pd.Timestamp.now()
        data = s.run(sql)
        costTime = (pd.Timestamp.now() - startTime) / np.timedelta64(1, 'ms')
        cost += costTime
        count += len(data)
    countQue.put(count)
    costQue.put(cost)
def main():
    for parallel in parallels:
        # 计划查询总次数
        countQueMax = parallel*sqlNum
        #print("并发查询进程数是{0}，单个进程的循环查询次数是{1}，计划查询总次数是{2}。".format(parallel, sqlNum, countQueMax))
        sql = """
        pt = loadTable('{db}', '{tb}')
        select max(TradePrice) from pt where date(TradeTime) = 2023.02.01 group by SecurityID""".format(db = dbName, tb = tbName)
        taskList = []
        for i in range(parallel):
            testTask = Process(target=testSQL, args=("Process" + str(i), sql))
            taskList.append(testTask)
        startTime = pd.Timestamp.now()
        #print("开始执行并行查询的任务的时间是{0}".format(startTime))
        for p in taskList:
            p.start()
        for p in taskList:
            p.join()
        endTime = pd.Timestamp.now()
        sumCount = []
        while countQue.empty() == False:
            data = countQue.get()
            sumCount.append(data)
        sumCount = np.sum(sumCount)
        sumCost = []
        while costQue.empty() == False:
            data = costQue.get()
            sumCost.append(data)
        sumCost = np.sum(sumCost)
        #print("结束执行并行查询的任务的时间是{0}".format(endTime))
        #print("并发查询任务总耗时是{0}毫秒".format((endTime - startTime) / np.timedelta64(1, 'ms')))
        #print("总查询记录数是{0}".format(sumCount))
        #print("查询平均用时是{0}".format(sumCost / sqlNum / parallel))
        #print("并发查询性能RPS是{0}".format(sumCount/((endTime - startTime) / np.timedelta64(1, 's'))))
        #print("并发查询性能QPS是{0}".format(countQueMax/((endTime - startTime) / np.timedelta64(1, 's'))))
        #print("每次查询的平均耗时是{0}毫秒".format(((endTime - startTime) / np.timedelta64(1, 'ms'))/countQueMax))
        #print("end")

In [349]:
#4.运行查询函数
if __name__ == '__main__':
    main()